In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

/tmp/ipykernel_1421/956813560.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
#chack for updated files\n,
%load_ext autoreload
%autoreload 2
#import packages
from __future__ import division

import numpy as np
import glob, json
import pickle
import os as os_pack
import matplotlib.pyplot as plt
import corner
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import healpy as hp
import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.hypermodel import HyperModel #issue here
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
#import enterprise_wavelets as models
from enterprise.signals.deterministic_signals import Deterministic
from enterprise.signals.parameter import function
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import importlib
import re
from QuickBurst import QuickBurst_MCMC as QuickBurst_MCMC

Optional mpi4py package is not installed.  MPI support is not available.


In [3]:
parDir = '../20_pulsars/WN_only/Pars/'
timDir =  "../20_pulsars/WN_only/Tims/"

parfiles = sorted(glob.glob(parDir + '*.par'))
timfiles = sorted(glob.glob(timDir + "*.tim"))

psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t, ephem=None, clk=None)
    psrs.append(psr)

[tempo2Util.C:396] Warning: [TIM1] Please place MODE flags in the parameter file 
[tempo2Util.C:401] Warning: [DUP1] duplicated warnings have been suppressed.


In [ ]:
with open("/home/user/.../data.pkl", 'rb') as f:
    psrs = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '/home/user/.../data.pkl'

FileNotFoundError: [Errno 2] No such file or directory: '/home/user/.../data.pkl'

In [4]:
noise_file_sim = "../20_pulsars/WN_only/noise_dict.json"
with open(noise_file_sim, 'rb') as h:
    noise_params = json.load(h)
noise_params

{'JPSR00_efac': 1.8689379530521901,
 'JPSR01_efac': 2.059675826005349,
 'JPSR02_efac': 3.0748606945657295,
 'JPSR03_efac': 2.585580282428298,
 'JPSR04_efac': 1.306306353450894,
 'JPSR05_efac': 2.2773544810789987,
 'JPSR06_efac': 4.549380171029036,
 'JPSR07_efac': 4.219982666682523,
 'JPSR08_efac': 4.203481666382409,
 'JPSR09_efac': 2.0239824772557204,
 'JPSR10_efac': 4.8229400301325756,
 'JPSR11_efac': 1.8810720005787385,
 'JPSR12_efac': 2.7899938334974146,
 'JPSR13_efac': 2.239301276926412,
 'JPSR14_efac': 2.0720756835008314,
 'JPSR15_efac': 4.601274822894665,
 'JPSR16_efac': 2.1540371144986725,
 'JPSR17_efac': 3.32976374621823,
 'JPSR18_efac': 3.154554728980668,
 'JPSR19_efac': 4.022119621532103,
 'JPSR00_log10_t2equad': -5.969194645533057,
 'JPSR01_log10_t2equad': -6.685017179105907,
 'JPSR02_log10_t2equad': -5.626366465579119,
 'JPSR03_log10_t2equad': -7.718808126457086,
 'JPSR04_log10_t2equad': -5.670995915028478,
 'JPSR05_log10_t2equad': -7.2278321569508135,
 'JPSR06_log10_t2equa

In [5]:
#Setting dataset max time and reference time
maximum = 0
minimum = np.inf
for psr in psrs:
    if psr.toas.max() > maximum:
        maximum = psr.toas.max()
    if psr.toas.min() < minimum:
        minimum = psr.toas.min()


#Sets reference time
tref = minimum
print(tref/3600/24/365)
t0_max = (maximum - minimum)/365/24/3600
print(t0_max)

145.20546846866188
9.945234130667314


In [6]:
#Number of shape parameter updates
N_slow=int(1e5)

#How often to update fisher matrix proposals (based on shape parameter updates)
n_fish_update = int(N_slow/2)

#Ratio of projection parameter updates per shape parameter update
projection_updates = 1

#Number of samples to thin (based on total samples N_slow*projection_updates)
thinning = 1

T_max = 4 #2
n_chain = 5 #3

#Prior bounds on shape params
tau_min = 0.2
tau_max = 3.0 #5.0
f_max = 1e-7
f_min = 1e-8 #3.5e-9

#Load in tau scan proposal files
ts_file = "./20_simple_psrs_data/wavelet_tau_scan.pkl"
glitch_ts_file = "./20_simple_psrs_data/glitch_tau_scan.pkl"

filepath = "./20_simple_psrs_data/"
os.makedirs(filepath, exist_ok = True)
savepath = filepath + "20_simple_psrs_run" #NOTE: DO NOT ADD FILE EXTENSION
importlib.reload(QuickBurst_MCMC)
samples, acc_fraction, swap_record, rj_record, ptas, log_likelihood, betas, PT_acc = QuickBurst_MCMC.run_qb(N_slow, T_max, n_chain, psrs,
                                                                    max_n_wavelet=0,
                                                                    min_n_wavelet=0,
                                                                    n_wavelet_start=0,
                                                                    RJ_weight=0,
                                                                    glitch_RJ_weight=0,
                                                                    regular_weight=2,
                                                                    noise_jump_weight=2,
                                                                    PT_swap_weight=2,
                                                                    tau_scan_proposal_weight=0,
                                                                    glitch_tau_scan_proposal_weight=0,
                                                                    tau_scan_file=ts_file,
                                                                    glitch_tau_scan_file=glitch_ts_file,
                                                                    #gwb_log_amp_range=[-18,-15],
                                                                    rn_log_amp_range=[-18,-11],
                                                                    wavelet_log_amp_range=[-10.0,-5.0],
                                                                    per_psr_rn_log_amp_range=[-18,-11],
                                                                    #rn_params = [noise_params['gw_crn_log10_A'],noise_params['gw_crn_gamma']],
                                                                    prior_recovery=False,
                                                                    #gwb_amp_prior='log-uniform',
                                                                    rn_amp_prior='log-uniform',
                                                                    wavelet_amp_prior='uniform',
                                                                    per_psr_rn_amp_prior='log-uniform',
                                                                    #gwb_on_prior=0.975,
                                                                    max_n_glitch=0,
                                                                    #n_glitch_start='random',
                                                                    glitch_log_amp_range=[-10.0,-5.0],
                                                                    glitch_amp_prior='uniform',
                                                                    f0_max = f_max,
                                                                    f0_min = f_min,
                                                                    tau_max_in = tau_max,
                                                                    tau_min_in = tau_min,
                                                                    t0_max=t0_max,
                                                                    tref = tref,
                                                                    vary_white_noise=True,  
                                                                    include_rn=False, vary_rn=False,
                                                                    include_equad=True,
                                                                    include_ecorr=False,
                                                                    include_efac=True,
                                                                    wn_backend_selection=False,
                                                                    noisedict= noise_params,
                                                                    include_per_psr_rn=False,
                                                                    vary_per_psr_rn=False,
                                                                    #resume_from=savepath,
                                                                    #per_psr_rn_start_file=RN_start_file,
                                                                    n_fish_update = n_fish_update,
                                                                    savepath=savepath, save_every_n=100,
                                                                    n_fast_to_slow=projection_updates, thin = thinning)

Saving every 100 samples, total samples: 100000  

Ending total saved samples: 100000 

Number of params per pulsar:  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Number of pta params:  40
Using 5 temperature chains with a geometric spacing of 1.414.     Temperature ladder is:
 [1.         1.41421356 2.         2.82842712 4.        ]
Using 5 temperature chains with custom spacing:  [1.         1.41421356 2.         2.82842712 4.        ]
Dynamic temperature adjustment: OFF
-----
42
40
40
-----
Initial beta (1/T) ladder is:
 [1.         0.70710678 0.5        0.35355339 0.25      ]
param list:  ['JPSR00_efac', 'JPSR00_log10_t2equad', 'JPSR01_efac', 'JPSR01_log10_t2equad', 'JPSR02_efac', 'JPSR02_log10_t2equad', 'JPSR03_efac', 'JPSR03_log10_t2equad', 'JPSR04_efac', 'JPSR04_log10_t2equad', 'JPSR05_efac', 'JPSR05_log10_t2equad', 'JPSR06_efac', 'JPSR06_log10_t2equad', 'JPSR07_efac', 'JPSR07_log10_t2equad', 'JPSR08_efac', 'JPSR08_log10_t2equad', 'JPSR09_efac', 'JPSR09_log10_t2e

/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_MCMC.py:392: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
/home/mitch/miniconda3/envs/QuickBurst/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.


QB logl calc for initial sample
param list:  ['JPSR00_efac', 'JPSR00_log10_t2equad', 'JPSR01_efac', 'JPSR01_log10_t2equad', 'JPSR02_efac', 'JPSR02_log10_t2equad', 'JPSR03_efac', 'JPSR03_log10_t2equad', 'JPSR04_efac', 'JPSR04_log10_t2equad', 'JPSR05_efac', 'JPSR05_log10_t2equad', 'JPSR06_efac', 'JPSR06_log10_t2equad', 'JPSR07_efac', 'JPSR07_log10_t2equad', 'JPSR08_efac', 'JPSR08_log10_t2equad', 'JPSR09_efac', 'JPSR09_log10_t2equad', 'JPSR10_efac', 'JPSR10_log10_t2equad', 'JPSR11_efac', 'JPSR11_log10_t2equad', 'JPSR12_efac', 'JPSR12_log10_t2equad', 'JPSR13_efac', 'JPSR13_log10_t2equad', 'JPSR14_efac', 'JPSR14_log10_t2equad', 'JPSR15_efac', 'JPSR15_log10_t2equad', 'JPSR16_efac', 'JPSR16_log10_t2equad', 'JPSR17_efac', 'JPSR17_log10_t2equad', 'JPSR18_efac', 'JPSR18_log10_t2equad', 'JPSR19_efac', 'JPSR19_log10_t2equad']
first_sample:  [ 1.86893795 -5.96919465  2.05967583 -6.68501718  3.07486069 -5.62636647
  2.58558028 -7.71880813  1.30630635 -5.67099592  2.27735448 -7.22783216
  4.54938017 

/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_MCMC.py:577: RuntimeWarning: invalid value encountered in true_divide
/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_MCMC.py:715: RuntimeWarning: Mean of empty slice
/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_MCMC.py:756: RuntimeWarning: invalid value encountered in true_divide


Changed some nan elements in the Fisher matrix to 1.0
N_Noise_Params_changed:  4
eign_per_pulsar re-written
eign_per_pulsar re-written
eign_per_pulsar re-written
eign_per_pulsar re-written
eign_per_pulsar re-written


/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_MCMC.py:715: RuntimeWarning: Mean of empty slice


Saving at sample 100
int(stop_iter) - 1:  99999
Path exists?  False
Resume_from == savefile?:  False
Progress: 0.10% Acceptance fraction #columns: chain number; rows: proposal type (glitch_RJ, glitch_tauscan, wavelet_RJ, wavelet_tauscan, PT, fast_jump, regular_jump, noise_jump):

Run Time: 1.7846832275390625s
[[       nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan]
 [0.36666667 0.36666667 1.         1.                nan]
 [       nan        nan        nan        nan        nan]
 [0.         0.         0.         0.         0.        ]
 [0.96551724 0.82758621 0.93103448 0.89655172 0.89655172]]
[0.35264198 0.36056671 0.97817474 0.98652093]


/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_MCMC.py:756: RuntimeWarning: invalid value encountered in true_divide


Saving at sample 200
int(stop_iter) - 1:  99999
Progress: 0.20% Acceptance fraction #columns: chain number; rows: proposal type (glitch_RJ, glitch_tauscan, wavelet_RJ, wavelet_tauscan, PT, fast_jump, regular_jump, noise_jump):

Run Time: 3.2358293533325195s
[[       nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan]
 [0.18032787 0.19672131 0.50819672 0.98360656        nan]
 [       nan        nan        nan        nan        nan]
 [0.         0.         0.         0.         0.        ]
 [0.91304348 0.82608696 0.85507246 0.91304348 0.85507246]]
[0.17343048 0.19379772 0.50454174 0.97085036]
Saving at sample 300
int(stop_iter) - 1:  99999
Progress: 0.30% Acceptance fraction #columns: chain number; rows: proposal type (glitch_RJ, glitch_tauscan, wavelet_RJ, wavelet_tauscan, PT, fast_jump, regular_jump, noise_jump):

Run

/home/mitch/miniconda3/envs/QuickBurst/lib/python3.9/site-packages/enterprise/signals/signal_base.py:1121: RuntimeWarning: divide by zero encountered in true_divide
/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_lnlike.py:345: RuntimeWarning: divide by zero encountered in true_divide
/home/mitch/miniconda3/envs/QuickBurst/lib/python3.9/site-packages/enterprise/signals/signal_base.py:1119: RuntimeWarning: divide by zero encountered in true_divide
/home/mitch/miniconda3/envs/QuickBurst/lib/python3.9/site-packages/enterprise/signals/signal_base.py:1125: RuntimeWarning: divide by zero encountered in log
/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_lnlike.py:672: RuntimeWarning: invalid value encountered in matmul
/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_lnlike.py:673: RuntimeWarning: invalid value encountered in matmul
/home/mitch/test_quick_burst/quick_burst/QuickBurst/QuickBurst_MCMC.py:879: RuntimeWarning: invalid value encountered